In [2]:
!pip install --upgrade pip
!pip install gensim

     |████████████████████████████████| 24.2 MB 179 kB/s eta 0:00:01
     |████████████████████████████████| 122 kB 82.5 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-2.2.1-py3-none-any.whl size=114113 sha256=e4e7d76b0c21f9bd45f4548a75e3c9925260ae153c75183ba10c2de6c2a6e764
  Stored in directory: /home/ec2-user/.cache/pip/wheels/12/6f/d1/6b06b392acb738f552f25dcfe0aac54a20ec854a20f55240aa
Successfully built smart-open
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [47]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import re
from src import utils, nmf_utils, nlp_topic_utils
from bs4 import BeautifulSoup

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import string

from gensim.corpora.dictionary import Dictionary
from gensim.models import Word2Vec

from collections import Counter
from operator import itemgetter
import itertools

from functools import reduce

In [50]:
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')

In [51]:
contracted_words_dict = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "I would",
    "i'd've": "I would have",
    "i'll": "I will",
    "i'll've": "I will have",
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'alls": "you alls",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had",
    "you'd've": "you would have",
    "you'll": "you you will",
    "you'll've": "you you will have",
    "you're": "you are",
    "you've": "you have"
}

In [52]:
addtl_stop_words = ['still', 'got', 'guy', 'say', 'still', 'thing', 'us', 'will', 'get', 'ok', 'we', 'please', 'would', 'could', 'im']

stop_words = stopwords.words("english")#.append(append_to_stopwords)
stop_words = stop_words + addtl_stop_words
stop_words_dict = Counter(stop_words)

# List of punctuation
addtl_punc = ['...']
punc = list(set(string.punctuation))
punc = punc + addtl_punc

# Word2Vec

In [5]:
wallstreet = pd.read_csv('data/wallstreet_master.csv')

In [53]:
wallstreet['all_words_prep'] = wallstreet['processed_text'].apply(lambda x: nlp_topic_utils.process_text(x, contracted_words_dict, punc, stop_words_dict))

In [65]:
all_words_arr = wallstreet['all_words_prep'].tolist()
all_words = list(itertools.chain.from_iterable(all_words_arr))

In [67]:
all_words[:20]

['thought',
 'introduce',
 'new',
 'community',
 'trying',
 'learn',
 'much',
 'figured',
 'may',
 'well',
 'start',
 'far',
 'like',
 'simplicity',
 'site',
 'hope',
 'go',
 'far',
 'searching',
 'tip']

In [68]:
min_count = 2

word2vec = Word2Vec(all_words, min_count=min_count)

In [69]:
vocab = word2vec.wv.vocab
print(vocab)

{'t': <gensim.models.keyedvectors.Vocab object at 0x7efe28778cf8>, 'h': <gensim.models.keyedvectors.Vocab object at 0x7efe28778d68>, 'o': <gensim.models.keyedvectors.Vocab object at 0x7efe28778208>, 'u': <gensim.models.keyedvectors.Vocab object at 0x7efe28778e80>, 'g': <gensim.models.keyedvectors.Vocab object at 0x7efe287781d0>, 'i': <gensim.models.keyedvectors.Vocab object at 0x7efe28778c88>, 'n': <gensim.models.keyedvectors.Vocab object at 0x7efe28778e10>, 'r': <gensim.models.keyedvectors.Vocab object at 0x7efe28778d30>, 'd': <gensim.models.keyedvectors.Vocab object at 0x7efe28778f60>, 'c': <gensim.models.keyedvectors.Vocab object at 0x7efe28778cc0>, 'e': <gensim.models.keyedvectors.Vocab object at 0x7efe28778e48>, 'w': <gensim.models.keyedvectors.Vocab object at 0x7efe28778c50>, 'm': <gensim.models.keyedvectors.Vocab object at 0x7efe28778eb8>, 'y': <gensim.models.keyedvectors.Vocab object at 0x7efe28778ba8>, 'l': <gensim.models.keyedvectors.Vocab object at 0x7efe28778f98>, 'a': <gen

In [ ]:
# View "changelog" as a vector.

v1 = word2vec.wv['changelog']
print(v1)